In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import os

# Clean every dataframes and merge

## Marathon Lausanne 2016

In [ ]:
DATA_DIR = './Data/Lausanne_Marathon_2016'
!ls -hl './Data/Lausanne_Marathon_2016'

In [ ]:
def clean_dataframe(df, fields=['catégorie', 'rang', 'nom', 'an', 'lieu', 'équipe', 'pénalité', 'temps', 'retard', 'acode'], to_numeric_column = ['rang', 'pénalité']):

    # Drop last row
    dataframe = df.drop(df.index[len(df)-1])[fields]

    # Parse column in numeric
    for column in to_numeric_column:
        if column in dataframe.columns:
            dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce')

    for row in dataframe.itertuples():
        
        # Fix issue in overall and moyenne columns
        if 'overall' in fields:
            # Split last column
            split = str(row.moyenne).split('. ', 1)
            if 'moyenne' in fields and len(split) > 1:
                dataframe.set_value(row.Index, 'overall', split[0])
                dataframe.set_value(row.Index, 'moyenne', split[1])
            else:
                dataframe.set_value(row.Index, 'overall', np.nan)

        # Remove () in doss column
        if 'doss' in fields and  str(row.doss)[0] == '(' and str(row.doss)[len(row.doss) - 1] == ')':
            dataframe.set_value(row.Index, 'doss', row.doss[1:-1])

        # Parse time
        if 'temps' in fields:
            try:
                dataframe.set_value(row.Index, 'temps', pd.to_datetime(row.temps, format='%H:%M.%S,%f'))
            except ValueError:
                dataframe.set_value(row.Index, 'temps', pd.to_datetime(row.temps, format='%M.%S,%f', errors='coerce'))
        if 'retard' in fields:
            try:
                dataframe.set_value(row.Index, 'retard', pd.to_datetime(row.retard, format='%H:%M.%S,%f'))
            except ValueError:
                dataframe.set_value(row.Index, 'retard', pd.to_datetime(row.retard, format='%M.%S,%f', errors='coerce'))
        if 'an' in fields:
            dataframe.set_value(row.Index, 'an', pd.to_datetime(row.an, format='%Y.0', errors='coerce'))

    return dataframe

### Runners: A

In [ ]:
!head ./Data/Lausanne_Marathon_2016/services.datasport.com_2016_lauf_lamara_ALFAA.HTM.csv

In [ ]:
df_2016_lauf_lamara_ALFAA = pd.read_csv(DATA_DIR + '/services.datasport.com_2016_lauf_lamara_ALFAA.HTM.csv', index_col=0)
df_2016_lauf_lamara_ALFAA.head()

In [ ]:
clean_dataframe(df_2016_lauf_lamara_ALFAA).head()

### All runners

In [ ]:
dataframes = []
for filename in os.listdir(DATA_DIR):
    print('Read and clean: ' + filename)
    uncleaned_df = pd.read_csv(DATA_DIR + '/' + filename, index_col=0)
    dataframes.append(clean_dataframe(uncleaned_df))

In [ ]:
merged_df = pd.concat(dataframes)
merged_df.head()

In [ ]:
merged_df.to_csv('./Data/Lausanne_Marathon_2016.csv')
merged_df.to_pickle('./Data/Lausanne_Marathon_2016.pickle')

In [ ]:
!head './Data/Lausanne_Marathon_2016.csv'

## All Marathon Lausanne

In [ ]:
lauf_directories = [
    'Lausanne_Marathon_2016',
    'Lausanne_Marathon_2015',
    'Lausanne_Marathon_2014',
    'Lausanne_Marathon_2013',
    'Lausanne_Marathon_2012',
    'Lausanne_Marathon_2011',
    'Lausanne_Marathon_2010',
    'Lausanne_Marathon_2009',
    'Lausanne_Marathon_2005',
    'Lausanne_Marathon_2004',
    'Lausanne_Marathon_2003',
    'Lausanne_Marathon_2002',
    'Lausanne_Marathon_2001',
    'Lausanne_Marathon_2000',
    'Lausanne_Marathon_2008',
    'Lausanne_Marathon_2007',
    'Lausanne_Marathon_2006',
    'Lausanne_Marathon_1999',
]

for directory in lauf_directories:
    directory_path = './Data/' + directory
    print('Start ' + directory_path)
    dataframes = []
    for filename in os.listdir(directory_path):
        print('    Read and clean: ' + filename)
        uncleaned_df = pd.read_csv(directory_path + '/' + filename, index_col=0)
        fields = [column for column in uncleaned_df.columns if column in ['catégorie', 'rang', 'nom', 'an', 'lieu', 'équipe', 'pénalité', 'temps', 'retard', 'acode']]
        to_numeric_column = [column for column in ['rang', 'pénalité'] if column in fields]
        dataframes.append(clean_dataframe(uncleaned_df, fields, to_numeric_column))
    merged_df = pd.concat(dataframes)
    merged_df.to_csv(directory_path + '.csv')
    merged_df.to_pickle(directory_path + '.pickle')
    print('End ' + directory_path + '\n')